In [1]:
import requests
import re
from bs4 import BeautifulSoup

In [2]:
facebook_session = requests.session()

facebook_session.headers.update({
        'User-Agent': 'Mozilla/5.0 (X11; Linux i686; rv:39.0) Gecko/20100101 Firefox/39.0'
})

In [3]:
facebook_email = 'luizrodrigo46@hotmail.com'
facebook_password = 'luiz05012015'

In [4]:
reponse = facebook_session.get('https://m.facebook.com')

In [5]:
response = facebook_session.post('https://m.facebook.com/login.php', data={
    'email': facebook_email,
    'pass': facebook_password
}, allow_redirects=False)

In [6]:
home_page = facebook_session.get('https://m.facebook.com/home.php')

In [7]:
home_parser = BeautifulSoup(home_page.content, 'html.parser')

In [8]:
home_link = home_parser.find_all('a')

In [9]:
for link in home_link:
    if link.text == 'Perfil':
        print(link.get('href'))
        perfil_page = facebook_session.get("https://m.facebook.com{0}".format(link.get('href')))
        perfil_parser = BeautifulSoup(perfil_page.content, 'html.parser')
        break

/luiz.rodrigo.357?ref_component=mbasic_home_header&ref_page=%2Fwap%2Fhome.php&refid=7


In [10]:
years_links = perfil_parser.find_all('a', {'href':re.compile('yearSectionsYears')})

In [35]:
for year_link in years_links:
    print("--------->SCRAPPING ALL PUBLICATIONS OF {}".format(year_link.text))
    # enter in the year publications page
    year_page = facebook_session.get("https://m.facebook.com{0}".format(year_link.get('href')))
    year_parser = BeautifulSoup(year_page.content, 'html.parser')


    see_more = 1
    
    while see_more:
        pubs = year_parser.find_all('a', text='História completa') # get all page publications
        
        for pub in pubs:
            # enter in the publication page detail
            pub_page = facebook_session.get("https://m.facebook.com{0}".format(pub.get('href')))
            pub_parser = BeautifulSoup(pub_page.content, 'html.parser')
                
            print("--------->SCRAPED ONE PUBLICATION: {}".format(pub_parser.find('abbr').text))
            
            # get the reactions link
            reactions = pub_parser.find('a', {'href':re.compile('/ufi/reaction/profile/browser/')})
            if not reactions.text:
                reaction_type = "None"
                reaction_user = "None"
                print("{}:{}".format(reaction_user, reaction_type))
                
            
            if reactions:
                reactions_link = reactions.get('href')
                
                # enter in the publication reactions list
                reactions_page = facebook_session.get("https://m.facebook.com{0}".format(reactions_link))
                reactions_parse = BeautifulSoup(reactions_page.content, 'html.parser')
                
                reactions_see_more = 1
                while reactions_see_more:                
                    for reaction in reactions_parse.find_all('li'):
                        # get the reaction type and the user that make this reaction
                        try:
                            reaction_type = reaction.img.next_element('img')[0].get('alt')
                            reaction_user = reaction.img.get('alt')
                            print("{}:{}".format(reaction_user, reaction_type))
                            
                        except:
                            pass
                    
                    reactions_see_more_parser = reactions_parse.find('a', text='Ver mais')
                    if reactions_see_more_parser:
                        reactions_link = reactions_see_more_parser.get('href')
                        reactions_page = facebook_session.get("https://m.facebook.com{0}".format(reactions_link))
                        reactions_parse = BeautifulSoup(reactions_page.content, 'html.parser')
                    else:
                        reactions_see_more = 0

                    #raise NotImplementedError

            see_more_parser = year_parser.find('a', text='Mostrar mais')
            if see_more_parser:
                see_more_link = see_more_parser.get('href')
                year_page = facebook_session.get("https://m.facebook.com{0}".format(see_more_link))
                year_parser = BeautifulSoup(year_page.content, 'html.parser')
            else:
                see_more = 0

--------->SCRAPPING ALL PUBLICATIONS OF 2017
--------->SCRAPED ONE PUBLICATION: 21 h
Luiz Roma:Curtir
Bernard Lyra:Curtir
Lucas Maxciel:Haha
--------->SCRAPED ONE PUBLICATION: 19 de julho às 20:28
Stephane Vale:Amei
--------->SCRAPED ONE PUBLICATION: 16 de julho às 22:25
None:None
--------->SCRAPED ONE PUBLICATION: 16 de julho às 20:37
Athyrson Machado Ribeiro:Curtir
Jefferson Wherculis:Curtir
Lucas Maxciel:Curtir
Tchesco França:Curtir
--------->SCRAPED ONE PUBLICATION: 16 de julho às 18:13
Isaac Guedelho:Curtir
Stephane Vale:Amei
Eron Araujo Dutra:Curtir
Athyrson Machado Ribeiro:Curtir
Mariene Sales:Curtir
Marluce Maciel:Curtir
Tatiane Morais:Curtir
Verônica Coelho:Curtir
Rodrigo Andrade:Curtir
Cida Gorniak:Curtir
Leonardo Mavignier:Curtir
Leidiane Vale:Curtir
Rebecka Maddox:Curtir
Francisca Aguiar:Curtir
Lucas Rodriguez:Curtir
Sabrina Maciel:Curtir
Fernando Fera:Curtir
João Neto:Curtir
Patrick Mazulo:Amei
--------->SCRAPED ONE PUBLICATION: 3 de junho às 18:17
Luiz Thiago Coelho:Haha


KeyboardInterrupt: 